---
author: Zeel B Patel
badges: true
categories:
  - ML
  - CV
date: "2024-02-10"
description: Compare your performance with a random baseline.
title: Object Detection Random Baseline
toc: true
---


# Imports


In [62]:
import os
import numpy as np
import supervision as sv
from roboflow import Roboflow
from dotenv import load_dotenv
load_dotenv()

True